In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
import owlready2
import rdflib
from map import ResourceMapper
from graph import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')
rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)

In [4]:
user_query = '용감한 형제가 작곡한 노래의 장르는?'
resource_combination = rm.process(user_query)
conceptual_graph = cgg.generate_conceptual_graph(resource_combination)
query_graph = qgg.generate_query_graph(conceptual_graph)

In [5]:
for score, qg in query_graph:
    print(qg)

print(f'\n: 총 {len(query_graph)}개의 query graph candidates generated')

[[('foaf:Person', 'skmo:isComposedBy', 'mo:Track')], [('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]]

: 총 1개의 query graph candidates generated


In [6]:
final_query_graph = sorted(query_graph)[0][1]
final_query_graph

[[('foaf:Person', 'skmo:isComposedBy', 'mo:Track')],
 [('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]]

In [72]:
query = """
SELECT ?target
WHERE
"""
query += '{\n'
for query_arc in final_query_graph:
    for s,p,o in query_arc:
        query += (' ').join([s,p,o]) + '\n'
query += '}'
print(query)


SELECT ?target
WHERE
{
foaf:Person skmo:isComposedBy mo:Track
mo:Track skmo:hasGenre skmo:Genre
}


In [69]:
g = rdflib.Graph()
g.parse(config['owl']['path'])

SKMO = rdflib.Namespace("http://www.sktelecom.com/skmo/")
g.bind("skmo", SKMO)

knows_query = """
SELECT DISTINCT ?target
WHERE {
    ?x skmo:isComposedBy skmo:용감한_형제;
    ?x skmo:hasGenre ?target
}"""

triple_list = []
qres = g.query(knows_query)

for row in qres:
    print(row.target)

http://www.sktelecom.com/skmo/k-pop
http://www.sktelecom.com/skmo/R&B
http://www.sktelecom.com/skmo/hip_hop
